<a href="https://colab.research.google.com/github/deltorobarba/science/blob/main/pasqal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantum Computing with Pasqal**

In [1]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq
import cirq_pasqal
from cirq_pasqal import ThreeDQubit, TwoDQubit, PasqalVirtualDevice, PasqalNoiseModel

installing cirq...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.7 MB/s eta 0:00:00
installed cirq.


In [2]:
width = 6
height = 6
depth = 1
# Creating p_qubits, a list of ThreeDQubits.
p_qubits = [
    ThreeDQubit(row, col, layer)
    for row in range(width)
    for col in range(height)
    for layer in range(depth)
]

In [3]:
p_qubits = TwoDQubit.square(6)  # 6x6 square array of TwoDQubits

In [4]:
# Initialize and create a circuit
initial_circuit = cirq.Circuit()
initial_circuit.append(cirq.CZ(p_qubits[0], p_qubits[1]))
initial_circuit.append(cirq.Z(p_qubits[0]))
initial_circuit.append(cirq.CX(p_qubits[0], p_qubits[2]))

# Create a Pasqal device with a control radius of 2.1 (in units of the lattice spacing)
p_device = PasqalVirtualDevice(control_radius=2.1, qubits=p_qubits)

# Validate the circuit using the device
try:
    p_device.validate_circuit(initial_circuit)
except ValueError as e:
    # Uh oh!  This circuit does not pass validation.
    print(e)

cirq.CNOT is not a supported gate


In [5]:
pasqal_gateset = cirq_pasqal.PasqalGateset(include_additional_controlled_ops=False)
pasqal_circuit = cirq.optimize_for_target_gateset(initial_circuit, gateset=pasqal_gateset)

# TODO(https://github.com/quantumlib/Cirq/issues/6655) - remove after fixup
pasqal_circuit = cirq.Circuit(pasqal_circuit.all_operations(), strategy=cirq.InsertStrategy.NEW)

print(pasqal_circuit)

# Now the circuit validates correctly!
p_device.validate_circuit(pasqal_circuit)

(0, 0): ───@───Z──────────────────@───────────────────Z─────────
           │                      │
(1, 0): ───@──────────────────────┼─────────────────────────────
                                  │
(2, 0): ───────────PhX(0.5)^0.5───@───PhX(-0.5)^0.5───────Z^0───


In [6]:
initial_circuit = cirq.Circuit()
initial_circuit.append(cirq.H(p_qubits[0]))
initial_circuit.append(cirq.H(p_qubits[1]))
print('Initial Circuit, does not pass validation:')
print(initial_circuit)
try:
    p_device.validate_circuit(initial_circuit)
except ValueError as e:
    # Uh oh!  This circuit does not pass validation.
    print(e)

pasqal_circuit = cirq.Circuit(initial_circuit.all_operations(), strategy=cirq.InsertStrategy.NEW)
print('')
print('Converted Circuit, passes validation:')
print(pasqal_circuit)

# Now the circuit validates correctly!
p_device.validate_circuit(pasqal_circuit)

Initial Circuit, does not pass validation:
(0, 0): ───H───

(1, 0): ───H───
Cannot do simultaneous gates. Use cirq.InsertStrategy.NEW.

Converted Circuit, passes validation:
(0, 0): ───H───────

(1, 0): ───────H───


In [7]:
# It is impossible to add a multi-qubit gate if the qubits involved are too far away
try:
    p_device.validate_operation(cirq.CZ(p_qubits[0], p_qubits[-1]))
except ValueError as msg:
    print("ERROR:", msg)
else:
    print("Failed to raise the expected ValueError.")

ERROR: Qubits pasqal.TwoDQubit(0, 0), pasqal.TwoDQubit(5, 5) are too far away


In [8]:
import numpy as np

# Place qubits at vertices of an equilateral triangle
qs = [TwoDQubit(0, 0), TwoDQubit(1, 0), TwoDQubit(0.5, np.sqrt(3) / 2)]
q_meas = [TwoDQubit(1, 0), TwoDQubit(0, 0)]


def state_preparation():
    for q in qs[:-1]:
        yield cirq.H(q)
    yield cirq.X(qs[-1])
    yield cirq.H(qs[-1])


def oracle():
    # Signals the state 10
    yield cirq.X(qs[0])
    yield cirq.CCX(*qs)
    yield cirq.X(qs[0])


def grover_operator():
    for q in qs[:-1]:
        yield cirq.H(q)
        yield cirq.X(q)

    yield cirq.CZ(*qs[:-1])

    for q in qs[:-1]:
        yield cirq.X(q)
        yield cirq.H(q)


def generate_grover():
    yield state_preparation()
    yield oracle()
    yield grover_operator()


def generate_pasqal_grover():
    pasqal_gateset = cirq_pasqal.PasqalGateset(include_additional_controlled_ops=False)
    grover_cicruit = cirq.optimize_for_target_gateset(
        cirq.Circuit(generate_grover()), gateset=pasqal_gateset
    )
    return cirq.Circuit(grover_cicruit.all_operations(), strategy=cirq.InsertStrategy.NEW)


device = PasqalVirtualDevice(control_radius=1.1, qubits=qs)
grover_circuit = generate_pasqal_grover()
device.validate_circuit(grover_circuit)
grover_circuit.append(cirq.measure(*q_meas, key='x'))

In [9]:
# To execute on one of Pasqal's servers, get an access token and uncomment these lines
sampler = cirq_pasqal.PasqalSampler(remote_host='http://34.98.71.118/v0/pasqal',
                                    access_token='addhere')

In [ ]:
# or locally simulate
#sampler = cirq.Simulator()
#data_raw = sampler.run(grover_circuit, repetitions=200)

In [ ]:
import matplotlib.pyplot as plt

# Create dictionary of results
data0 = data_raw.data.to_dict()
vec = [j for j in range(4)]
res = np.zeros(len(vec))
for jj in range(200):
    number = data0['x'][jj]
    res[number] += 1
res = res / 200.0

# Display plot of states
fig = plt.figure()
ax = fig.add_axes([0.16, 0.16, 0.78, 0.78])
ax.plot(vec, res, 'o')
ax.set_xticks(vec)
ax.set_xticklabels([r'$|00\rangle$', r'$|01\rangle$', r'$|10\rangle$', r'$|11\rangle$'])
plt.xlabel(r'$j$', fontsize=20)
plt.ylabel(r'$p$', fontsize=20)
plt.show()

https://quantumai.google/cirq/hardware/pasqal/getting_started